In [4]:
import json
import math
import random
from collections import defaultdict
from itertools import count

import numpy as np
import gymnasium as gym
from gymnasium.spaces import Discrete
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt

import games

## DQN

In [6]:
env = gym.make('games/BlackjackEnv')

NameNotFound: Environment `BlackjackEnv` doesn't exist in namespace games. Did you mean: `BlackJack`?